In [ ]:
# initialise libraries 
import pandas as pd
import requests

Step 1: Perform API call to perform JSON request


In [ ]:
response = requests.get("https://api.data.gov.sg/v1/transport/taxi-availability")
taxi_json = response.json()
taxi_json

{'type': 'FeatureCollection',
 'crs': {'type': 'link',
  'properties': {'href': 'http://spatialreference.org/ref/epsg/4326/ogcwkt/',
   'type': 'ogcwkt'}},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPoint',
    'coordinates': [[103.6396, 1.32],
     [103.664081, 1.301968],
     [103.688599, 1.34313033333333],
     [103.688916166667, 1.34110483333333],
     [103.69084235, 1.3456374],
     [103.692320533333, 1.34596735],
     [103.69248785, 1.33789496666667],
     [103.69346, 1.34],
     [103.69397, 1.3686],
     [103.69593, 1.33785],
     [103.696114, 1.347454],
     [103.696201566667, 1.33971436666667],
     [103.696369, 1.341739],
     [103.69673, 1.35],
     [103.69727, 1.35],
     [103.697430916667, 1.35048471666667],
     [103.69908, 1.34],
     [103.700293733333, 1.33848925],
     [103.7003855, 1.3502565],
     [103.700482516667, 1.33937596666667],
     [103.70416, 1.34],
     [103.7041671, 1.33955445],
     [103.7060305, 1.3363345],
     [103.706627533333, 1.

2. Convert JSON request to dataframe (ie: table form)

In [ ]:
pd_taxi_json = pd.json_normalize(taxi_json)
#extract only the features part in JSON
taxi_features = taxi_json['features']
#convert features tag to dataframe
pd_taxi_features = pd.json_normalize(taxi_features)
pd_taxi_features.head()

,type,geometry.type,geometry.coordinates,properties.timestamp,properties.taxi_count,properties.api_info.status
0,Feature,MultiPoint,"[[103.6396, 1.32], [103.664081, 1.301968], [10...",2023-03-21T14:13:48+08:00,1008,healthy


In [ ]:
# Generate date range. Reduced to 1 day interval
list_jan2023 = pd.date_range(start='1/1/2023', end='1/2/2023', freq='5T')
# create list containing dates and time
list_jan2023_date = []
list_jan2023_hour = []
list_jan2023_min = []
for datetime in list_jan2023:
    str_date = str(datetime)
    list_jan2023_date.append(str_date[:10] + "T")
    list_jan2023_hour.append(str_date[11:13] + "%3A")
    list_jan2023_min.append(str_date[14:16] + "%3A00")
list_jan2023_zip = [str1+str2+str3 for str1,str2,str3 in zip(list_jan2023_date, list_jan2023_hour, list_jan2023_min)]
# save the new features into zip
list_jan2023_new = list(list_jan2023_zip)


In [ ]:
import time

# declare the base URL
base_url = "https://api.data.gov.sg/v1/transport/taxi-availability?date_time="
# declare the empty list
list_features = []

t0 = time.time()
for datetime in list_jan2023_new:

    response = requests.get(base_url + datetime)
    
    taxi_json = response.json()
    #taxi_json
    taxi_features = pd.json_normalize(taxi_json['features'])
    
    list_features.append(taxi_features)

t1 = time.time()

print(f"{t1-t0} seconds to download 500 urls and convert json")

336.3903262615204 seconds to download 500 urls and convert json


In [ ]:
df = pd.concat(list_features)
df['time'] = list_jan2023
df.info()
df.to_csv("taxi.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289 entries, 0 to 0
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   type                        289 non-null    object        
 1   geometry.type               289 non-null    object        
 2   geometry.coordinates        289 non-null    object        
 3   properties.timestamp        289 non-null    object        
 4   properties.taxi_count       289 non-null    int64         
 5   properties.api_info.status  289 non-null    object        
 6   time                        289 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 18.1+ KB
